<a href="https://colab.research.google.com/github/breeNep/mineriaDatos/blob/main/MD_1910_06PLN_Casiari_Analis_exploratorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pickle

In [ ]:
dtm_df = pd.read_pickle('dtm.pkl')
dtm_df

In [ ]:
data = dtm_df.transpose()
data

In [ ]:
top_words_per_year = pd.DataFrame()
for year in data.columns:
    top_words = data[year].nlargest(10)
    top_words_per_year = pd.concat([top_words_per_year, top_words], axis =1)

top_words_per_year.columns = data.columns
top_words_per_year

In [ ]:
words = [word for year in top_words_per_year.columns for word in top_words_per_year[year].dropna().index]
word_counts = Counter(words)
word_counts

In [ ]:
sorted_word_counts = dict(sorted(word_counts.items(), key=lambda item: item[1], reverse=True))

labels = list(sorted_word_counts.keys())
values = list(sorted_word_counts.values())
indices = np.arange(len(labels))

plt.figure(figsize=(10,5))
plt.bar(indices, values, color='skyblue')
plt.xlabel("Palabras")
plt.ylabel("Frecuencia")
plt.title("Frecuencia de las palabras mas comunes")
plt.xticks(indices, labels, rotation='vertical')
plt.show()

In [ ]:
corpus_df = pd.read_pickle('corpus.pkl')
corpus_df

In [ ]:
words_to_find = ["casa", "mundo","tiempo", "vida"]
cfd = nltk.ConditionalFreqDist(
    (word, index)
    for index, row in corpus_df.iterrows()
    for word in nltk.word_tokenize(row['transcript'].lower())
    if word in words_to_find

)
cfd.plot()
plt.show()

In [ ]:
stop_words = pickle.load(open("stop_words.pkl", "rb"))
data_clean = pd.read_pickle("data_clean.pkl")
data_clean

In [ ]:
wc = WordCloud(stopwords=stop_words, background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)
plt.rcParams['figure.figsize'] = [16,12]

for index, year in enumerate(corpus_df.index):
  wc.generate(corpus_df.transcript[year])
  plt.subplot(4,3, index+1)
  plt.imshow(wc, interpolation="bilinear")
  plt.axis("off")
  plt.title(year)

  plt.show()

In [ ]:
from pyparsing import results
posts_per_year = [50, 27, 18, 50, 42, 22, 50, 33, 31, 17, 33, 13]
results_df = pd.DataFrame(index=data_clean.index)

results_df['total_words'] = data_clean['transcript'].apply(lambda x: len(x.split()))
results_df['unique_words'] = data_clean['transcript'].apply(lambda x: len(set(x.split())))
results_df['post_per_year'] = posts_per_year
results_df['words_per_post'] = (results_df['total_words']/ results_df['post_per_year']).round()
results_df

In [ ]:
fig, axs=plt.subplots(2,2, figsize=(15,10))
axs = axs.ravel()
for i, column in enumerate(results_df.columns):
    axs[i].plot(results_df.index, results_df[column], marker='o', linestyle='--', color='b')
    axs[i].set_title(column.replace("_", " ").title())
    axs[i].set_xlabel(column.replace("_", " ").title())
    axs[i].set_xlabel("Year")
    axs[i].set_ylabel(column.replace("_", " ").title())

    for j, txt in enumerate(results_df[column]):
        axs[i].annotate(txt,(results_df.index[j], results_df[column][j]))

plt.tight_layout()
plt.show()